# Build a model for LSTM Neural Network

In [50]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
#!pip install keras-metrics

In [51]:
import tqdm
import numpy as np
import tensorflow
import keras_metrics # for recall and precision metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dropout, Dense
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pickle

from keras.preprocessing.text import Tokenizer

In [52]:
SEQUENCE_LENGTH = 100 # the length of all sequences (number of words per sample)
EMBEDDING_SIZE = 100  # Using 100-Dimensional GloVe embedding vectors
TEST_SIZE = 0.20 # ratio of testing set

BATCH_SIZE = 64# number of epochs

# to convert labels to integers and vice-versa
#label2int = {"truthful": 0, "deceptive": 1} #added
#int2label = {0: "truthful", 1: "deceptive"} #added

label2int = {'Neutral':0, 'Positive':1, 'Negative':2, 'Extremely Positive':3,'Extremely Negative':4} #added
int2label = {0:'Neutral', 1:'Positive', 2:'Negative', 3:'Extremely Positive',4:'Extremely Negative'} #added

In [53]:
import pandas as pd

In [54]:
data=pd.read_csv("Corona_NLP_train.csv",encoding='latin-1')                         

In [55]:
data.isnull().sum()

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [56]:
data.shape

(41157, 6)

In [57]:
df=data.dropna()

In [58]:
df.shape

(32567, 6)

In [59]:
df.isnull().sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [60]:
df.Sentiment.unique()

array(['Neutral', 'Positive', 'Negative', 'Extremely Positive',
       'Extremely Negative'], dtype=object)

In [61]:
x=df['OriginalTweet']

In [62]:
x

0        @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1        advice Talk to your neighbours family to excha...
2        Coronavirus Australia: Woolworths to give elde...
5        As news of the regionÂs first confirmed COVID...
6        Cashier at grocery store was sharing his insig...
                               ...                        
41147    YÂall really shitting that much more at home?...
41149    Still shocked by the number of #Toronto superm...
41150    I never that weÂd be in a situation &amp; wor...
41152    Airline pilots offering to stock supermarket s...
41156    @TartiiCat Well new/used Rift S are going for ...
Name: OriginalTweet, Length: 32567, dtype: object

In [63]:
x=list(x)

In [64]:
x

['@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8',
 'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order',
 'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P',
 'As news of the regionÂ\x92s first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, @Tim_Dodson reports https://t.co/cfXch7a2lU',
 'Cashier at grocery store was sharing his insights on #Covid_19 To prove his credibility he commented "I\'m in Civics class so I know what I\'m talking about". https://t.co/ieFDNeHgDO',
 "Was at the supermarket today. Didn't buy toilet paper. #Reb

In [65]:
df['Sentiment'] = df['Sentiment'].map({'Neutral':'Neutral', 'Positive':'Positive', 'Negative':'Negative', 'Extremely Positive': 'Positive', 'Extremely Negative': 'Negative'})

In [66]:
df['Sentiment']

0         Neutral
1        Positive
2        Positive
5        Positive
6        Positive
           ...   
41147    Negative
41149    Negative
41150    Positive
41152     Neutral
41156    Negative
Name: Sentiment, Length: 32567, dtype: object

In [73]:
df['Sentiment'] = df['Sentiment'].replace({'Neutral':0, 'Positive':1, 'Negative':2})

In [74]:
y=df['Sentiment']

In [75]:
y.unique()

array([0, 1, 2], dtype=int64)

In [76]:
y=list(y)

In [77]:
y

[0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 0,


In [78]:
type(y)

list

In [79]:
# Text tokenization
# vectorizing text, turning each text into sequence of integers
tokenizer = Tokenizer(num_words=None,char_level=True,oov_token='UNK')
tokenizer.fit_on_texts(x)

In [80]:

# convert to sequence of integers
x = tokenizer.texts_to_sequences(x)


In [81]:
print(x[0])

[37, 17, 3, 10, 20, 9, 23, 8, 3, 2, 37, 16, 12, 8, 13, 53, 18, 6, 12, 6, 10, 2, 37, 11, 12, 9, 8, 7, 8, 5, 21, 2, 12, 5, 5, 16, 7, 31, 26, 26, 5, 28, 11, 4, 26, 8, 19, 40, 32, 19, 6, 10, 41, 16, 6, 2, 6, 10, 14, 2, 12, 5, 5, 16, 7, 31, 26, 26, 5, 28, 11, 4, 26, 35, 35, 50, 18, 12, 18, 19, 40, 11, 11, 2, 6, 10, 14, 2, 12, 5, 5, 16, 7, 31, 26, 26, 5, 28, 11, 4, 26, 8, 41, 10, 13, 40, 14, 35, 10, 4, 52]


In [82]:
# convert to numpy arrays
x = np.array(x)
y = np.array(y)
# pad sequences at the beginning of each sequence with 0's
# for example if SEQUENCE_LENGTH=4:
# [[5, 3, 2], [5, 1, 2, 3], [3, 4]]
# will be transformed to:
# [[0, 5, 3, 2], [5, 1, 2, 3], [0, 0, 3, 4]]
x = pad_sequences(x, maxlen=SEQUENCE_LENGTH)

In [84]:
# One Hot encoding labels
# [spam, ham, spam, ham, ham] will be converted to:
# [1, 0, 1, 0, 1] and then to:
# [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]

#y = [ label2int[label] for label in y ]
y = to_categorical(y)

In [85]:
 print(y)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [86]:
#### split and shuffle
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=7)

In [49]:
#ar=get_embedding_vectors(tokenizer)


100

In [87]:
def get_embedding_vectors(tokenizer, dim=100):
    embedding_index = {}
    with open(f"glove.6B.{dim}d.txt", encoding='utf8') as f:
        for line in tqdm.tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = vectors

    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index)+1, dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found will be 0s
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [89]:
#embedding_matrix = get_embedding_vectors(tokenizer)
embedding_matrix[5]

Reading GloVe: 400000it [00:17, 23106.66it/s]


array([ 0.13482   ,  0.40224001, -0.42265999, -0.055631  , -0.55742002,
        0.043634  ,  0.049172  ,  0.17382   , -0.74579   , -0.11306   ,
        0.19373   , -0.0612    , -0.04722   ,  0.61628997,  0.16717   ,
        0.18415   ,  0.17518   ,  0.032346  ,  0.87703001, -0.29756001,
        0.47646001, -0.031404  ,  0.36258999,  0.50297999,  0.20622   ,
        1.13629997,  0.039542  , -0.26769999,  0.15408   , -0.77929002,
        1.15670002,  1.42219996, -0.21405999, -0.081809  ,  0.53378999,
        0.069233  ,  0.18365   ,  0.21626   ,  0.18494999,  0.51840001,
        1.26250005, -0.39466   ,  0.0044504 , -0.61466002, -0.86791998,
        0.17854001, -0.64734   ,  0.92421001,  0.10692   , -0.50519001,
        0.27298999,  0.053007  ,  0.36574   ,  0.013988  , -0.41345999,
       -1.70550001, -0.67447001,  0.62946999,  1.62720001,  0.37147   ,
       -0.52477002, -0.21547   , -0.50581998, -0.33827001,  0.63836998,
        0.90878999,  0.51358998,  0.61179   ,  0.47222   ,  0.36

In [33]:
import tensorflow as tf

In [90]:
def get_model(tokenizer, lstm_units):
    """
    Constructs the model,
    Embedding vectors => LSTM => 2 output Fully-Connected neurons with softmax activation
    """
    # get the GloVe embedding vectors
    embedding_matrix = get_embedding_vectors(tokenizer)
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index)+1,
              EMBEDDING_SIZE,
              weights=[embedding_matrix],
              trainable=False,
              input_length=SEQUENCE_LENGTH))

    model.add(LSTM(lstm_units, recurrent_dropout=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation="softmax"))
    # compile as rmsprop optimizer
    # aswell as with recall metric
    model.compile(optimizer="adam", loss="categorical_crossentropy",
                  metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    model.summary()
    return model

In [99]:
# constructs the model with 128 LSTM units
model = get_model(tokenizer=tokenizer, lstm_units=128)

Reading GloVe: 400000it [00:18, 21221.70it/s]


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          11600     
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 387       
                                                                 
Total params: 129,235
Trainable params: 117,635
Non-trainable params: 11,600
_________________________________________________________________


In [36]:
# initialize our ModelCheckpoint and TensorBoard callbacks
# model checkpoint for saving best weights
model_checkpoint = ModelCheckpoint("Covid_19_Sentiment_{val_loss:.2f}", save_best_only=True,verbose=1)
# for better visualization
tensorboard = TensorBoard(f"Covid_19_Sentiment_{time.time()}")
# print our data shapes
print("X_train.shape:", X_train.shape)
print("X_test.shape:", X_test.shape)
print("y_train.shape:", y_train.shape)
print("y_test.shape:", y_test.shape)
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=32, epochs=100,verbose=1)

X_train.shape: (26053, 100)
X_test.shape: (6514, 100)
y_train.shape: (26053, 3)
y_test.shape: (6514, 3)
Epoch 1/100
815/815 [==============================] - 77s 93ms/step - loss: 1.0501 - accuracy: 0.4297 - precision: 0.4516 - recall: 0.0437 - val_loss: 1.0379 - val_accuracy: 0.4440 - val_precision: 0.7000 - val_recall: 0.0011
Epoch 2/100
815/815 [==============================] - 79s 96ms/step - loss: 1.0437 - accuracy: 0.4375 - precision: 0.4861 - recall: 0.0187 - val_loss: 1.0380 - val_accuracy: 0.4450 - val_precision: 0.4899 - val_recall: 0.0224
Epoch 3/100
815/815 [==============================] - 80s 98ms/step - loss: 1.0375 - accuracy: 0.4375 - precision: 0.4843 - recall: 0.0190 - val_loss: 1.0175 - val_accuracy: 0.4487 - val_precision: 0.5397 - val_recall: 0.0157
Epoch 4/100
815/815 [==============================] - 82s 100ms/step - loss: 1.0216 - accuracy: 0.4487 - precision: 0.4893 - recall: 0.0403 - val_loss: 1.0156 - val_accuracy: 0.4383 - val_precision: 0.5052 - val_re

Epoch 72/100
815/815 [==============================] - 89s 109ms/step - loss: 0.6064 - accuracy: 0.7395 - precision: 0.7851 - recall: 0.6833 - val_loss: 1.1293 - val_accuracy: 0.5450 - val_precision: 0.5776 - val_recall: 0.4863
Epoch 73/100
815/815 [==============================] - 91s 111ms/step - loss: 0.6018 - accuracy: 0.7459 - precision: 0.7876 - recall: 0.6882 - val_loss: 1.0971 - val_accuracy: 0.5467 - val_precision: 0.5803 - val_recall: 0.4883
Epoch 74/100
815/815 [==============================] - 92s 113ms/step - loss: 0.6109 - accuracy: 0.7362 - precision: 0.7822 - recall: 0.6795 - val_loss: 1.0844 - val_accuracy: 0.5456 - val_precision: 0.5764 - val_recall: 0.4831
Epoch 75/100
815/815 [==============================] - 94s 115ms/step - loss: 0.6001 - accuracy: 0.7456 - precision: 0.7872 - recall: 0.6894 - val_loss: 1.1357 - val_accuracy: 0.5500 - val_precision: 0.5797 - val_recall: 0.5006
Epoch 76/100
815/815 [==============================] - 89s 109ms/step - loss: 0.598

In [37]:
# get the loss and metrics
result = model.evaluate(X_test, y_test)
# extract those
loss = result[0]
accuracy = result[1]
precision = result[2]
recall = result[3]

204/204 [==============================] - 3s 15ms/step - loss: 1.1190 - accuracy: 0.5296 - precision: 0.5621 - recall: 0.4725


In [39]:
def get_predictions(text):
    sequence = tokenizer.texts_to_sequences([text])
    # pad the sequence
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction
    prediction = model.predict(sequence)[0]
    # one-hot encoded vector, revert using np.argmax
    return int2label[np.argmax(prediction)]

In [42]:
text=str(input("enter the statement: "))

enter the statement: i wont wear a mask


In [43]:
print(get_predictions(text))

Negative


In [ ]:
#You will join one of Europeâ€™s most hotly tipped startups with plenty of opportunities to grow and the chance to be part of our little revolution.Â This role lasts for 6 months with a possibility to become permanent depending on performance.Â You are invited to join our company holidaysÂ (which are completely insane), 2 days holiday per month, boxing yoga + our team lunches every Friday.Â Oh, and coffee, thereâ€™s plenty of coffee. Just one last thing,Â you can use the office Sauna whenever you want too.#LDN

In [ ]:
text = "You will join one of Europeâ€™s most hotly tipped startups with plenty of opportunities to grow and the chance to be part of our little revolution.Â This role lasts for 6 months with a possibility to become permanent depending on performance.Â You are invited to join our company holidaysÂ (which are completely insane), 2 days holiday per month, boxing yoga + our team lunches every Friday.Â Oh, and coffee, thereâ€™s plenty of coffee. Just one last thing,Â you can use the office Sauna whenever you want too.#LDN"
print(get_predictions(text))

In [ ]:
import h5py

In [ ]:
model.save('Covid_Tweet.h5')

In [92]:
from tensorflow.keras.models import load_model